In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb

In [ ]:
#Should include a call to GCP Big query#
#Install gsutil

In [2]:
#Load raw datasets
questions = context.catalog.load("questions",) 
lectures = context.catalog.load("lectures",)
#train =  context.catalog.load("train_filter",)

NameError: name 'context' is not defined

In [3]:
train = train[train.content_type_id == False]
#arrange by timestamp


train = train.sort_values(['timestamp'], ascending=True)

train.drop(['timestamp','content_type_id'], axis=1,   inplace=True)

results_c = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]

results_u = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']

In [4]:
#reading in question df
questions_df = pd.read_csv('data/01_raw/questions.csv',
                            usecols=[0,1, 3,4],
                            dtype={'question_id': 'int16',
                                   'part': 'int8',
                                   'bundle_id': 'int8',
                                   'tags': 'str'}
                                  )

tag = questions_df["tags"].str.split(" ", n = 10, expand = True) 
tag.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']

questions_df =  pd.concat([questions_df,tag],axis=1)
questions_df['tags1'] = pd.to_numeric(questions_df['tags1'], errors='coerce')
questions_df['tags2'] = pd.to_numeric(questions_df['tags2'], errors='coerce')
questions_df['tags3'] = pd.to_numeric(questions_df['tags3'], errors='coerce')
questions_df['tags4'] = pd.to_numeric(questions_df['tags4'], errors='coerce')
questions_df['tags5'] = pd.to_numeric(questions_df['tags5'], errors='coerce')
questions_df['tags6'] = pd.to_numeric(questions_df['tags6'], errors='coerce')

In [5]:
X = train.iloc[80000000:,:]
X['prior_question_had_explanation'].fillna(False, inplace=True)
X = pd.merge(X, results_u, on=['user_id'], how="left")
X = pd.merge(X, results_c, on=['content_id'], how="left")
X = pd.merge(X, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')

X=X[X.answered_correctly!= -1 ]
X=X.sort_values(['user_id'])
Y = X[["answered_correctly"]]
X = X.drop(["answered_correctly"], axis=1)

In [ ]:
lectures

In [6]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])
X.head()

,user_id,content_id,prior_question_elapsed_time,prior_question_had_explanation,answered_correctly_user,sum,answered_correctly_content,question_id,bundle_id,part,tags,tags1,tags2,tags3,tags4,tags5,tags6,prior_question_had_explanation_enc
2969848,13134,12263,10000.0,True,0.706356,878.0,0.570648,12263,-25,2,138 41 81 92,138.0,41.0,81.0,92.0,NaN,NaN,1
4342675,13134,12115,19000.0,True,0.706356,878.0,0.809202,12115,83,2,143 140 38 81,143.0,140.0,38.0,81.0,NaN,NaN,1
4342647,13134,12257,22000.0,True,0.706356,878.0,0.820975,12257,-31,2,90 100 92 29,90.0,100.0,92.0,29.0,NaN,NaN,1
4342609,13134,12236,18000.0,True,0.706356,878.0,0.720157,12236,-52,2,155 163 92 29,155.0,163.0,92.0,29.0,NaN,NaN,1
5217369,13134,13063,29666.0,True,0.706356,878.0,0.960396,13063,5,4,136 103 81,136.0,103.0,81.0,NaN,NaN,NaN,1


In [7]:
X = X[['answered_correctly_user', 'answered_correctly_content', 'sum','bundle_id','part','prior_question_elapsed_time','prior_question_had_explanation_enc','tags1','tags2','tags3']] 

X.fillna(0.5,  inplace=True)

In [8]:
train_ds = lgb.Dataset(X, Y,categorical_feature = ['part','tags1','tags2','tags3'])

In [9]:
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}


model_cv = lgb.cv(parameters,
                       train_ds,
                       nfold=10,
                       stratified=True,
                       num_boost_round=5000,
                       early_stopping_rounds=100,
                       return_cvbooster=True)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.115872 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.091193 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


In [1]:
#Plot metric mean value
plt.title("auc mean")
plt.plot(model_cv["auc-mean"])
plt.show()

/home/tdenimal/miniconda3/envs/riid_kaggle/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'plt' is not defined

In [2]:
#Plot metric mean value
plt.title("auc std")
plt.plot(model_cv["auc-std"])
plt.show()

NameError: name 'plt' is not defined



from  sklearn.model_selection import train_test_split
Xt, Xv, Yt, Yv = train_test_split(X, Y, test_size =0.2, shuffle=False)




lgb_train = lgb.Dataset(Xt, Yt,categorical_feature = ['part','tags1','tags2','tags3'])
lgb_eval = lgb.Dataset(Xv, Yv, reference=lgb_train,categorical_feature = ['part','tags1','tags2','tags3'])

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=10000,
    early_stopping_rounds=10
)